## This program inferring OCL by LLM4Models   

In [ ]:
# Choose 3 to abstract OCL from Java
# Choose 4 to abstract OCL from Python

What_I_Want=4

In [ ]:
import re
import torch
import time

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [ ]:
DEVICE="cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# choose version 1 or version 2 
if What_I_Want==3:
   peft_model_id='HA-Siala/Java-OCL-V1' # or
   #peft_model_id='HA-Siala/Java-OCL-V2' 
else:
   peft_model_id='HA-Siala/Python-OCL-V1' # or
   #peft_model_id='HA-Siala/Python-OCL-V2' 

In [ ]:
base_model_name = "mistralai/Mistral-7B-v0.3"  
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.padding_side = "left"  
tokenizer.pad_token = tokenizer.unk_token  

model = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.merge_and_unload()

model = AutoModelForCausalLM.from_pretrained(
   peft_model_id,  
   torch_dtype=torch.float16,
   device_map="auto"
)

In [ ]:
def GeneratePrompt(content):
   if What_I_Want==3:
      Instruction="""Generate an Object Constraint Language (OCL) specification for the provided Java code. The output should:
1. Ensure no repeated or redundant operations or classes.
2. Include only the OCL code for the provided Java code.
3. Do not include statements for items not found in the Java code."""
   else:
      Instruction="""Generate an Object Constraint Language (OCL) specification for the provided Python code. The output should:
1. Ensure no repeated or redundant operations or classes.
2. Include only the OCL code for the provided Python code.
3. Do not include statements for items not found in the Python code."""

   Prompt="""<s>[INST] Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately solves the following Task:""".strip()
   return f"""{Prompt} \n
### Instruction:
{Instruction}

### Input:
{content}
[/INST]

### Response:
"""

In [ ]:
def GenerateInferenceOutput(text): 
   tokenizer.pad_token = tokenizer.unk_token  
   inputs = tokenizer(GeneratePrompt(text), return_tensors="pt").to(DEVICE)
   inputs_length = len(inputs["input_ids"][0])
   with torch.inference_mode():
      outputs = model.generate(
         **inputs,  
         max_new_tokens= 8000, 
         temperature= 0.2, 
         do_sample= True, 
         pad_token_id=tokenizer.eos_token_id,
         top_p=0.9,
         use_cache=True,
      )
      output_p = tokenizer.batch_decode(outputs, skip_special_tokens=True)
      if output_p:
         output_text = output_p[0]
         split_text_p = output_text.split("Response:")
         return split_text_p[1].strip() if len(split_text_p) > 1 else None
      else:
         return None

In [ ]:
# Put the program here between """ and """
# You need to remove comments and empty lines (pre-processing)
content=""""""

In [ ]:
StartTime=time.time()
Output = GenerateInferenceOutput(content)
EndTime=time.time()
InferenceTime=EndTime - StartTime
print(Output)
print()
print(f"Inference Time: {InferenceTime:.2f} seconds")